## Multiple Chain 快速入门

Runnables 可以轻松地用来串联多个 Chains，使用 RunnablePassthrough 将输出同时传给多条后继链。

```
     Input
      / \
     /   \
 Chain1 Chain2
     \   /
      \ /
      Combine
```

本指南展示如何使用 Runnable 实现多个 AI 关于相同话题的辩论：

```
    输入话题
       |
       |
    原始观点
      / |\
     /  |  \
 正面论述| 反面论述
     \  | /
      \ |/
     最终总结
```

In [27]:
# 导入相关模块，包括运算符、输出解析器、聊天模板、ChatOpenAI 和 运行器
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_zhipu import ChatZhipuAI
from langchain_core.runnables import RunnablePassthrough

# 创建一个计划器，生成一个关于给定输入的论证
planner = (
    ChatPromptTemplate.from_template("生成关于以下内容的论点: {input}")
    # | ChatOpenAI()
    | ChatZhipuAI(model="glm-4-flash")
    | StrOutputParser()
    | {"base_response": RunnablePassthrough()}
)

# 创建正面论证的处理链，列出关于基础回应的正面或有利的方面
arguments_for = (
    ChatPromptTemplate.from_template(
        "列出关于{base_response}的正面或有利的方面"
    )
    # | ChatOpenAI()
    | ChatZhipuAI(model="glm-4-flash")
    | StrOutputParser()
)

# 创建反面论证的处理链，列出关于基础回应的反面或不利的方面
arguments_against = (
    ChatPromptTemplate.from_template(
        "列出关于{base_response}的反面或不利的方面"
    )
    # | ChatOpenAI()
    | ChatZhipuAI(model="glm-4-flash")
    | StrOutputParser()
)

# 创建最终响应者，综合原始回应和正反论点生成最终的回应
final_responder = (
    ChatPromptTemplate.from_messages(
        [
            ("ai", "{original_response}"),
            ("human", "正面观点:\n{results_1}\n\n反面观点:\n{results_2}"),
            ("system", "给出批评后生成最终回应"),
        ]
    )
    # | ChatOpenAI()
    | ChatZhipuAI(model="glm-4-flash")
    | StrOutputParser()
)
# 构建完整的处理链，从生成论点到列出正反论点，再到生成最终回应
chain = (
    planner
    | {
        "results_1": arguments_for,
        "results_2": arguments_against,
        "original_response": itemgetter("base_response"),
    }
    | final_responder
)

In [5]:
print(chain.invoke({"input": "房地产低迷"}))

'正面观点补充：\n\n一、论据一：房地产低迷的调整和优化（续）\n\n3. **促进创新与竞争**：房地产低迷时期，企业可能面临更大的生存压力，这会促使它们寻求创新和改进，提高服务质量，同时也可能激发新的市场参与者进入，增加市场竞争力。\n\n二、论据二：房地产低迷的启示和反思（续）\n\n2. **消费观念转变**：房地产低迷可能促使消费者更加注重长期价值和可持续性，而非短期投机，这有助于形成更为成熟和理性的消费文化。\n\n三、论据三：房地产低迷的社会效益（续）\n\n3. **环境改善**：房地产低迷可能会减少对土地资源的过度开发，有助于保护自然环境，改善城市生态环境。\n\n四、论据四：房地产低迷的金融稳定效应（续）\n\n2. **政策工具的多样化**：房地产低迷可能迫使政府探索更多样化的政策工具，如发展租赁市场、推动共有产权住房等，以应对房地产市场波动。\n\n反面观点补充：\n\n一、房地产低迷可能带来的负面影响（续）\n\n7. **房地产市场不透明**：低迷的市场可能导致信息不透明，增加交易风险，损害消费者权益。\n\n8. **房地产市场依赖性**：某些地区和城市可能过度依赖房地产市场作为经济增长的主要动力，这种依赖性可能导致经济脆弱性增加。\n\n二、刺激房地产市场的潜在不利影响（续）\n\n6. **房地产市场依赖性的加剧**：刺激房地产市场可能加剧对房地产的依赖，使得经济结构难以转型，长期来看不利于经济的健康发展。\n\n7. **房地产市场政策风险**：房地产市场政策的调整可能带来不确定性，如果政策变动过于频繁或力度不当，可能引发市场动荡。\n\n8. **房地产泡沫的再形成**：即便在低迷之后，过度的刺激措施也可能导致房地产泡沫的再次形成，进而带来更大的风险。'

#### 流式输出

In [ ]:
## chain 最终输出经过了 StrOutputParser 处理，所以可以直接输出流式输出 s
for s in chain.stream({"input": "全球经济"}):
    print(s, end="", flush=True)

正面观点的进一步细化：

1. **增强市场多样性**：全球经济一体化使得消费者能够接触到来自世界各地的产品和服务，增加了市场的多样性，满足了不同消费者的个性化需求。

2. **提升消费者福利**：由于全球范围内的价格竞争，消费者能够以更低的价格购买到高质量的商品和服务，从而提升了消费者的整体福利水平。

3. **优化产业链布局**：全球经济一体化促进了产业链的全球布局，企业可以根据各国的比较优势，优化生产流程，降低生产成本，提高全球产业链的效率和竞争力。

4. **加速政策改革**：为了适应全球经济一体化的要求，各国政府可能会加快经济和政策的改革步伐，以提升国内经济的国际竞争力。

5. **促进教育国际化**：全球经济一体化推动了教育资源的国际共享，使得更多人有机会接受国际化的教育，提高全球人才的整体素质。

6. **推动可持续发展**：在全球经济一体化的推动下，可持续发展理念得到更广泛的传播和实践，有助于实现经济增长与环境保护的平衡。

7. **增强政策透明度**：为了吸引外资和促进国际贸易，各国政府需要提高政策的透明度，这有助于防止腐败，提高政策执行的有效性。

8. **加强国际法治**：全球经济一体化促进了国际法治的发展，各国在贸易、投资和知识产权保护等方面更加遵守国际规则，有利于维护全球经济的稳定。

9. **提升全球治理能力**：全球经济一体化使得各国在处理全球性问题时更加紧密合作，提升了全球治理机构的决策效率和影响力。

10. **促进和平与安全**：全球经济一体化有助于减少国家间的冲突和对抗，通过经济合作促进和平与安全，为全球发展创造了稳定的环境。

反面观点的进一步细化：

1. **加剧收入

### Homework: 实现一个多链版本的代码生成，输入功能需求，输出2种（Python，Java）以上编程语言的代码实现。